In [1]:
from collections import Counter
import pandas as pd
import math
import numpy as np 
from sklearn.preprocessing import LabelEncoder 
from scipy.stats import poisson


We downloaded the "Hurricane" csv from the excel file mentioned in the question . Now we can apply changes as asked in the questions . 

* PART - A

In this part of the question we need to write program for t-test for correlation coefficient with a 1% level of significance between "“Max. sustained winds(mph)” and “Minimum pressure(mbar)”.

In [2]:
dataset = pd.read_csv('Assignment-1 - Hurricane.csv')

wind_speed = dataset['Max. sustained winds(mph)'] ;  

min_pressure = dataset['Minimum pressure(mbar)'] ; 

wind_speed = np.array(wind_speed) ; 
min_pressure = np.array(min_pressure) ; 
# print(wind_speed.shape) ; 
# print(wind_speed) ; 
# print(min_pressure.shape) ;  
# print(min_pressure) ;  


# Now we have got the column values in the np arrays . Now we can proceed with the t_test ; 




We will be using t_test to conduct t-test for correlation coefficient between “Max. sustained winds(mph)” and “Minimum pressure(mbar)” with 1% level of significance . Thus we formulate the hypothesis first : 

### Null Hypothesis (H₀)
There is no significant correlation between the two variables.

- H₀: ρ = 0

### Alternative Hypothesis (H₁)
There is a significant correlation between the two variables (two-tailed test).

- H₁: ρ ≠ 0

Let us first calculate the value of r (Pearson Correlation Coefficient) . 

In [3]:
# let r be the correlation coefficient . 

def calculate_mean(data) : 
  if (len(data) == 0):
    return None ; 
  
  return sum(data) / len(data) ;  


def calculate_num(wind_speed , min_pressure): 
  mean_wind_speed = calculate_mean(wind_speed) ; 
  
  mean_min_pressure = calculate_mean(min_pressure) ; 
  
  ans = 0 ; 
  
  for i in range(len(wind_speed)): 
    
    first_term = (wind_speed[i] - mean_wind_speed) ; 
    second_term = (min_pressure[i] -mean_min_pressure) ; 
    
    ans += first_term * second_term ; 
  
  return ans ; 

def calculate_den(wind_speed , min_pressure):
  mean_wind_speed = calculate_mean(wind_speed) ; 
  
  mean_min_pressure = calculate_mean(min_pressure) ; 
  
  ans = 0 ; 
  
  first_term = 0 ; 
  second_term = 0 ; 
  
  for i in range(len(wind_speed)):
    first_term += (wind_speed[i] - mean_wind_speed) ** 2 ; 
  
  for i in range(len(min_pressure)):
    second_term += (min_pressure[i] - mean_min_pressure) ** 2 ; 
  
  
  ans = first_term * second_term ; 
  
  ans = ans ** (1/2) ; 
  
  return ans ; 

numerator = calculate_num(wind_speed, min_pressure)
denominator = calculate_den(wind_speed, min_pressure)

if denominator != 0:  # To avoid division by zero
    r = numerator / denominator
else:
    r = None


print("The value of r is : " + str(r)) 


den_term = math.sqrt(1 - r * r) ; 

n = len(wind_speed) ; 

t = (r / den_term) * (math.sqrt(n - 2)) ; 

degree_of_freedom = (n - 2) ; 

print("The degree of freedom is: " + str(degree_of_freedom)) ; 
print("Value of t comes out to be: " + str(t)) ;  # we get the t-value from here . 

# Thus from the t-table shared to us , we can see that the value of alpha = 0.01 .(as the significance is 1% and the test is two tailed) . The value for n = 90 , cutoff = 2.632 and for n = 100 , cutoff = 2.626 . 

# Thus we know -5.49 < -2.632 , Thus we will reject the null hypothesis . 


The value of r is : -0.48359549563566406
The degree of freedom is: 99
Value of t comes out to be: -5.497270157344813


Thus from the t-table shared to us , we can see that the value of alpha = 0.01 .(as the significance is 1% and the test is two tailed) . The value for n = 90 , cutoff = 2.632 and for n = 100 , cutoff = 2.626 . 

Thus we know -5.49 < -2.632 , Thus we will reject the null hypothesis . 

# Part B

First we clean the data manually for such columns where there are garbage values in the month column . This way we make the dataset consistent and ready for use . I will be using one hot encoding for the categorical columns . 

We will add additional columns for the speed and the months as we are using one hot encoding thus changing them to numerical values of 0 and 1 . After this we will drop the categorical columns . We are also saving this new dataset as "Cleaned_hurricane.csv" . 

In [4]:
dataset_cleaned = pd.read_csv('Assignment-1 - Hurricane.csv')

wind_speed = dataset_cleaned['Max. sustained winds(mph)'] ; 
months = dataset_cleaned['Month']
# print(wind_speed.unique()) # they are only 5 in number 

# thus speed is also a categorical column and so is the months 

# thus first change the columns into one hot encodings 
 
unique_wind_speed = wind_speed.unique() ; 

unique_months = set()

for entry in months:
    unique_months.update(entry.split(", "))
  
for speed in unique_wind_speed:
    dataset_cleaned[f'Wind_{speed}mph'] = wind_speed.apply(lambda x: 1 if x == speed else 0)
    

for month in unique_months:
    dataset_cleaned[month] = months.apply(lambda x: 1 if month in x.split(", ") else 0)
  
dataset_cleaned = dataset_cleaned.drop(['Max. sustained winds(mph)', 'Month'], axis=1)

# print(dataset_cleaned)

dataset_cleaned.to_csv("Cleaned_hurricane.csv") 


# thus we successfully used one hot encodint in this dataset . Now we can move on forward . 

We can use chi squared testing for independence as both the columns are categorical columns . Thus we make the hypothesis as follows : 
## Hypotheses for Chi-Squared Test

### Null Hypothesis (H0)
The null hypothesis states that there is no association between the “Max. sustained winds(mph)” and the Months. In other words, the two variables are independent.

**H₀**: The “Max. sustained winds(mph)” is independent of the Months.

### Alternative Hypothesis (H1)
The alternative hypothesis states that there is an association between “Max. sustained winds(mph)” and the Months . This means that the two variables are dependent.

**H₁**: The“Max. sustained winds(mph)” is dependent on the month.

In [5]:
dataframe = pd.DataFrame(dataset_cleaned) ; 

wind_speed_cols = dataframe.filter(like='Wind_')
month_cols = ['August', 'November', 'July', 'October', 'September']

wind_speed_by_month = pd.DataFrame(0, index=wind_speed_cols.columns, columns=month_cols)


for i, row in dataframe.iterrows():
    
    for wind_col in wind_speed_cols.columns:
        if row[wind_col] == 1:  
            for month in month_cols:
                if row[month] == 1:  
                    wind_speed_by_month.loc[wind_col, month] += 1


print("Wind Speed by Month Table:")
print(wind_speed_by_month)


observed_data = wind_speed_by_month.values
row_sums = observed_data.sum(axis=1)
column_sums = observed_data.sum(axis=0)
total_sum = observed_data.sum()

expected = np.outer(row_sums, column_sums) / total_sum

chi_squared_val = ((observed_data - expected) ** 2 / expected).sum()

degree_of_freedom = (observed_data.shape[0] - 1) * (observed_data.shape[1] - 1)

print("\nChi-Squared Value:", chi_squared_val)
print("Degrees of Freedom:", degree_of_freedom)

# now the chi squared value for degree of freedom 12 and significance 5 % (as it is always a one sided tailed test) is : 21.026 . Thus , 7.971634778620072 < 21.026 so we will not reject the null hypothesis . 



Wind Speed by Month Table:
             August  November  July  October  September
Wind_150mph       8         0     0        5         11
Wind_140mph       9         1     1       10         18
Wind_155mph       4         1     0        2          6
Wind_130mph       7         0     0        5         16
Wind_145mph       7         1     0        8         17

Chi-Squared Value: 7.971634778620072
Degrees of Freedom: 16


Thus we get that 7.97 < 21.02 , thus we will not reject the null hypothesis . Thus , we conclude that “Max. sustained winds(mph)” is independent of month in this case

# Part C

In this we need to test with significance of 10% that if “Max. sustained winds(mph)” follows a Poisson distribution. 

## Hypotheses for Chi-Squared Goodness of Fit Test

**Null Hypothesis (\(H_0\))**: The data on "Max. sustained winds (mph)" follows a Poisson distribution with the estimated mean \( \lambda \).

**Alternative Hypothesis (\(H_1\)**: The data on "Max. sustained winds (mph)" does not follow a Poisson distribution.


Lets solve for the same using chi-square test for goodness of fit . 

Without Normalizing the Expected Frequencies to make sum of Expected Frequencies same as Observed Frequencies . 

In [6]:
wind_speed_data = dataset['Max. sustained winds(mph)'] ; 

wind_speed_data = np.array(wind_speed_data) ; 

# we will first calculate the freq of the wind_speed_data , this is the observed frequencies ; 

observed_frequencies = pd.Series(wind_speed_data).value_counts().sort_index() ; 

# now in the poisson distribution we will need to use the lambda value 

lambda_val = calculate_mean(wind_speed_data) ; 

total_count = len(wind_speed_data) ; 

# now we need to compute the expected frequencies (with we are considering as poisson distribution as said in the question) . 

def poisson_pmf(k, lambda_val):

    return poisson.pmf(k, lambda_val)

# We are making dictionary to store the expected frequencies 

expected_frequencies = {} ; 

for k in observed_frequencies.index: 
  expected_probablity = poisson_pmf(k , lambda_val) ; 
  
  expected_frequencies[k] = expected_probablity * total_count ; 


  # print(f"Wind Speed: {k}, Poisson Probability: {expected_probablity}, Expected Frequency: {expected_frequencies[k]}")

combined_vals = pd.DataFrame({'Observed' : observed_frequencies , 'Expected' : expected_frequencies}) 


chi_squared_val = ((combined_vals['Observed'] - combined_vals['Expected']) ** 2 / (combined_vals['Expected'])).sum() ; # this is formulae mentioned in the slides 

m = 1 ; 

degree_of_freedom = len(observed_frequencies) - 1 - m

print("The value of chi_squared_value is: " + str(chi_squared_val))

print("The value of degree of freedom is: " + str(degree_of_freedom)) ; 

# the value of chi-square for degree of freedom = 3 and significance = 0.1 (10 % ) is 6.251 . Now we see 631.92 > 6.251 . Thus we reject the null hypothesis . Thus we conclude that  The data on "Max. sustained winds (mph)" does not follow a Poisson distribution.

print(observed_frequencies) ; 
print(expected_frequencies) ; 


The value of chi_squared_value is: 631.9284580912764
The value of degree of freedom is: 3
130    23
140    28
145    24
150    18
155     8
Name: count, dtype: int64
{130: 2.1165926889411204, 140: 3.3610008729380514, 145: 3.232352525315237, 150: 2.6177813492202806, 155: 1.7955316595831554}


Thus we reject the null hypothesis . Thus we conclude that  The data on "Max. sustained winds (mph)" does not follow a Poisson distribution.

Normalizing the Expected Frequencies to make sum of Expected Frequencies same as Observed Frequencies . 

In [7]:
wind_speed_data = dataset['Max. sustained winds(mph)'] ; 

wind_speed_data = np.array(wind_speed_data) ; 

# we will first calculate the freq of the wind_speed_data , this is the observed frequencies ; 

observed_frequencies = pd.Series(wind_speed_data).value_counts().sort_index() ; 

# now in the poisson distribution we will need to use the lambda value 

lambda_val = calculate_mean(wind_speed_data) ; 

total_count = len(wind_speed_data) ; 

# now we need to compute the expected frequencies (with we are considering as poisson distribution as said in the question) . 

def poisson_pmf(k, lambda_val):

    return poisson.pmf(k, lambda_val)

# We are making dictionary to store the expected frequencies 

expected_frequencies = {} ; 

for k in observed_frequencies.index: 
  expected_probablity = poisson_pmf(k , lambda_val) ; 
  
  expected_frequencies[k] = expected_probablity * total_count ; 



  # print(f"Wind Speed: {k}, Poisson Probability: {expected_probablity}, Expected Frequency: {expected_frequencies[k]}")

total_observed = observed_frequencies.sum()
total_expected = sum(expected_frequencies.values())

if total_expected > 0:
    normalized_expected_frequencies = {k: (v / total_expected) * total_observed for k, v in expected_frequencies.items()}
else:
    normalized_expected_frequencies = expected_frequencies  # If no expected frequencies, keep original



combined_vals = pd.DataFrame({'Observed' : observed_frequencies , 'Expected' : normalized_expected_frequencies}) 


chi_squared_val = ((combined_vals['Observed'] - combined_vals['Expected']) ** 2 / (combined_vals['Expected'])).sum() ; # this is formulae mentioned in the slides 

m = 1 ; 

degree_of_freedom = len(observed_frequencies) - 1 - m

print("The value of chi_squared_value is: " + str(chi_squared_val))

print("The value of degree of freedom is: " + str(degree_of_freedom)) ; 

# the value of chi-square for degree of freedom = 3 and significance = 0.1 (10 % ) is 6.251 . Now we see 631.92 > 6.251 . Thus we reject the null hypothesis . Thus we conclude that  The data on "Max. sustained winds (mph)" does not follow a Poisson distribution.

print(observed_frequencies) ; 
print(expected_frequencies) ; 


The value of chi_squared_value is: 5.649893997595143
The value of degree of freedom is: 3
130    23
140    28
145    24
150    18
155     8
Name: count, dtype: int64
{130: 2.1165926889411204, 140: 3.3610008729380514, 145: 3.232352525315237, 150: 2.6177813492202806, 155: 1.7955316595831554}


In this case 5.64 < 6.251 . Thus we accept the null hypothesis in this case . This is the difference in the two cases i.e without normalizing and with normalizing . 